In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as cls
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits

dataset = load_digits()
X,y = dataset.data, dataset.target
for class_name, class_count in zip(dataset.target_names,np.bincount(dataset.target)):
    print(class_name, class_count)
# zip() 函数用于将可迭代的对象作为参数，将对象中对应的元素打包成一个个元组，然后返回由这些元组组成的列表。
# 如果各个迭代器的元素个数不一致，则返回列表长度与最短的对象相同，利用 * 号操作符，可以将元组解压为列表。
# np.bincount() Count number of occurrences of each value in array of non-negative ints.
# 显然是这里从0开始计算出现次数的，所以最后一个是最大值: 0出现几次，1出现几次，一直到9出现几次

0 178
1 182
2 177
3 183
4 181
5 182
6 181
7 179
8 174
9 180


In [3]:
y_binary_imbalanced = y.copy()
y_binary_imbalanced[y_binary_imbalanced !=1] = 0
print('Orinal labels:    ',y[:30])
print('New binary labels:',y_binary_imbalanced[:30])

Orinal labels:     [0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9]
New binary labels: [0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]


In [4]:
np.bincount(y_binary_imbalanced)

array([1615,  182])

In [5]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y_binary_imbalanced,random_state=0)
svm = SVC(kernel='rbf',C=1).fit(X_train,y_train)
svm.score(X_test,y_test)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.9088888888888889

## Dummy Classifiers

In [6]:
from sklearn.dummy import DummyClassifier
dummy_majority = DummyClassifier(strategy='most_frequent').fit(X_train,y_train)
y_dummy_predictions = dummy_majority.predict(X_test)
y_dummy_predictions
# most_frequent: 预测值是出现频率最高的类别
dummy_majority.score(X_test,y_test)

0.9044444444444445

In [7]:
svm = SVC(kernel='linear',C=1).fit(X_train,y_train)
svm.score(X_test,y_test)

0.9777777777777777

## confusion matrix

### binary confusion marix

In [8]:
from sklearn.metrics import confusion_matrix
dummy_majority = DummyClassifier(strategy='most_frequent').fit(X_train,y_train)
y_dummy_predictions = dummy_majority.predict(X_test)
confusion = confusion_matrix(y_test,y_dummy_predictions)
print('most frequent class (dummy classifier)\n',confusion)

most frequent class (dummy classifier)
 [[407   0]
 [ 43   0]]


In [9]:
dummy_classprop = DummyClassifier(strategy='stratified').fit(X_train,y_train)
y_classprop_predicted = dummy_classprop.predict(X_test)
confusion = confusion_matrix(y_test,y_classprop_predicted)
print('random class-proportional prediction (dummy classifier)\n',confusion)

random class-proportional prediction (dummy classifier)
 [[360  47]
 [ 40   3]]


In [10]:
svm = SVC(kernel='linear',C=1).fit(X_train,y_train)
svm_predicted = svm.predict(X_test)
confusion = confusion_matrix(y_test,svm_predicted)
print('SVM classifier (linear kernal, C=1)\n',confusion)

SVM classifier (linear kernal, C=1)
 [[402   5]
 [  5  38]]


In [11]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression().fit(X_train,y_train)
lr_predicted = lr.predict(X_test)
confusion = confusion_matrix(y_test,lr_predicted)
print('logistic regression (defalt settings)\n',confusion)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


logistic regression (defalt settings)
 [[401   6]
 [  6  37]]


In [12]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth=2).fit(X_train,y_train)
tree_predicted = dt.predict(X_test)
confusion = confusion_matrix(y_test,tree_predicted)
print('Decision Tree Classifier(max_depth=2)\n',confusion)

Decision Tree Classifier(max_depth=2)
 [[400   7]
 [ 17  26]]


## Evaluation metrics for binary classification

In [14]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
print('Accuracy: {:.2f}'.format(accuracy_score(y_test,tree_predicted)))
print('Precision: {:.2f}'.format(precision_score(y_test,tree_predicted)))
print('Recall: {:.2f}'.format(recall_score(y_test,tree_predicted)))
print('F1: {:.2f}'.format(f1_score(y_test,tree_predicted)))

Accuracy: 0.95
Precision: 0.79
Recall: 0.60
F1: 0.68


In [16]:
from sklearn.metrics import classification_report
print(classification_report(y_test,tree_predicted,target_names=['not 1','1']))
# support 是测试集中标签为true的数量

              precision    recall  f1-score   support

       not 1       0.96      0.98      0.97       407
           1       0.79      0.60      0.68        43

   micro avg       0.95      0.95      0.95       450
   macro avg       0.87      0.79      0.83       450
weighted avg       0.94      0.95      0.94       450



In [17]:
print('Random class-proportional (dummy)\n',
      classification_report(y_test,y_classprop_predicted,target_names=['not 1','1']))
print('SVM\n',
      classification_report(y_test,svm_predicted,target_names=['not 1','1']))
print('Logistic regression\n',
      classification_report(y_test,lr_predicted,target_names=['not 1','1']))
print('Decision tree\n',
      classification_report(y_test,tree_predicted,target_names=['not 1','1']))

Random class-proportional (dummy)
               precision    recall  f1-score   support

       not 1       0.90      0.88      0.89       407
           1       0.06      0.07      0.06        43

   micro avg       0.81      0.81      0.81       450
   macro avg       0.48      0.48      0.48       450
weighted avg       0.82      0.81      0.81       450

SVM
               precision    recall  f1-score   support

       not 1       0.99      0.99      0.99       407
           1       0.88      0.88      0.88        43

   micro avg       0.98      0.98      0.98       450
   macro avg       0.94      0.94      0.94       450
weighted avg       0.98      0.98      0.98       450

Logistic regression
               precision    recall  f1-score   support

       not 1       0.99      0.99      0.99       407
           1       0.86      0.86      0.86        43

   micro avg       0.97      0.97      0.97       450
   macro avg       0.92      0.92      0.92       450
weighted avg 

## Decison functions

In [18]:
X_train,X_test,y_train,y_test = train_test_split(X,y_binary_imbalanced,random_state=0)
y_scores_lr = lr.fit(X_train,y_train).decision_function(X_test)
y_score_list = list(zip(y_test[0:20],y_scores_lr[0:20]))
y_score_list

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[(0, -23.17711236290324),
 (0, -13.541470729541413),
 (0, -21.72290098969473),
 (0, -18.907438437430027),
 (0, -19.73582172900229),
 (0, -9.749807819560061),
 (1, 5.2349604859009276),
 (0, -19.307551661127864),
 (0, -25.101182889530396),
 (0, -21.82736239135058),
 (0, -24.151343401889438),
 (0, -19.576969790071697),
 (0, -22.574689400560423),
 (0, -10.823324268750714),
 (0, -11.912123406737392),
 (0, -10.97922371337485),
 (1, 11.206006114721543),
 (0, -27.64600231793191),
 (0, -12.859381428186682),
 (0, -25.848764845244997)]

## Multi-Class Evaluation

In [23]:
dataset = load_digits()
X,y = dataset.data,dataset.target
X_train_mc,X_test_mc,y_train_mc,y_test_mc = train_test_split(X,y,random_state=0)
svm = SVC(kernel='linear').fit(X_train_mc,y_train_mc)
svm_predicted_mc = svm.predict(X_test_mc)